In [22]:
from pathlib import PurePath

In [202]:
import sqlalchemy as sql
import pandas as pd
import numpy as np

In [3]:
from config import Config as cf

In [45]:
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_colwidth", 100)

----

In [8]:
from dotenv import dotenv_values
config = dotenv_values()

#### Download data

In [14]:
url = "postgresql+psycopg2://{}:{}@{}:{}/{}".format(
    config.get('DATABASE_LOGIN'),
    config.get('DATABASE_PASSWORD'),
    config.get('DATABASE_HOST'),
    config.get('DATABASE_PORT'),
    config.get('DATABASE_NAME')
)
engine = sql.create_engine(url=url)

with open("./sql/select_comments.sql", "r") as query:
    comments = pd.read_sql_query(query.read(), engine)

In [15]:
comments.query("not comment.isna() & not place_id.isna()", inplace=True)

In [16]:
with open("./sql/select_tags.sql", "r") as query:
    tags = pd.read_sql_query(query.read(), engine)

In [17]:
df = pd.DataFrame().from_records(comments.comment.values)

In [18]:
df.insert(0, "place_id", comments.place_id.values)

In [19]:
df = df.merge(tags, left_on="place_id", right_on="place_id", how="left")

In [20]:
df.drop("id", axis=1, inplace=True)

In [23]:
df.to_csv(PurePath(config.get('INITIAL_DATA_PATH'), "comments_df.csv"))

-----

### Making bi-gram`s

In [33]:
data = pd.read_csv(PurePath(config.get('INITIAL_DATA_PATH'), "comments_df.csv"), index_col=0)

In [34]:
tags = data.tag.unique()

In [217]:
data.head()

,place_id,review_title,review_body,review_date,review_rating,review_likes,review_tag,tag
0,1,Интересно прогуляться,Красивая набережная с барельефами отражающими историю города Тюмень Красивые виды Зимой строят с...,27 февраля 2023 г.,5.0,0,['RU'],Пирсы и пляжные променады
1,1,Место для романтических поогулок,Очень приятно прогуляться по набережной . Вечером красиво подсвечивается Мост Влюбленных и Кремл...,12 декабря 2022 г.,5.0,0,['RU'],Пирсы и пляжные променады
2,1,Концептуально-выдержанная идея вплоть до мелочей,"Отличное место для отдыха, прогулок и даже знакомства с историей города и Сибири. Очень круто, ч...",16 ноября 2022 г.,5.0,0,['RU'],Пирсы и пляжные променады
3,1,Набережная в Тюмени,"Красивая набережная, можно покататься на самокате, гуляет много народу, мост влюбленных вечером ...",2 сентября 2022 г.,5.0,1,['RU'],Пирсы и пляжные променады
4,1,Испортили хорошее место торгашами.,"Убрать всех приставучих предпринимателей! Пришли погулять вечером, отдохнуть, а там мадам с пито...",27 июля 2022 г.,1.0,1,"['С друзьями', 'RU']",Пирсы и пляжные променады


---

In [42]:
import re 

In [85]:
adjective = r"\w{4,}[ой, ий, ый, ая, ья, ью, ое, ые, ее, ье, ие, ьи, ого, его, ей, ему, ому, ыми, ими, ым, им, ом, ем, их, ых, ми]"

In [97]:
noun = r"\w{4,}[а, и, е, у, й, я, о, ом, ём, ем, ь, ью]"

In [118]:
regex =  adjective+noun

In [208]:
tags

array(['Пирсы и пляжные променады', 'Специализированные музеи', 'Мосты',
       'Образовательные объекты', 'Парки', 'Театры и представления',
       'Культурные объекты и достопримечательности', 'Памятники и статуи',
       'Церкви и соборы', 'Театры', 'Торговые центры',
       'Горячие источники и гейзеры', 'Аквапарки', 'Исторические музеи',
       'Арены и стадионы', 'Религиозные достопримечательности',
       'Оздоровительные клубы', 'Спа', 'Термальные спа-центры',
       'Зоопарки', 'Художественные галереи', 'Спортивные комплексы',
       'На Мельникайте в Городе мастеров наодится небльшой художественный салон Альбины Харитоновой, выбор картин большой, проводятся выставки.',
       'Сады', 'Боулинг', 'Бары и клубы', 'Комеди-клубы',
       'Клубы на пляже и у бассейна',
       'Другие виды активного отдыха на открытом воздухе',
       'Кулинарные мастер-классы',
       'Специализированные и сувенирные магазины', 'Кинотеатры',
       'Квест Рум', 'Концерты', 'Игровые и развлекательны

In [216]:
table = pd.ExcelWriter("./tables/tags_by_place.xlsx")

for tag in tags:

    df = data.query("tag == @tag").copy()
    
    place_dict = {}

    for i in df.index:

        review = (
            df
            .loc[i]
            .review_body
            .replace(",", '')
            .replace(".", " POI ")
            .lower()
        )
        review_rating = df.loc[i].review_rating
        findall = re.findall(
            regex, 
            review
        )
        for token in findall:
            if token not in place_dict:
                place_dict[token] = {
                    "count": 1, 
                    "ratings": [review_rating, ]
                }
            else:            
                count = final_dict[token]["count"]+1
                place_dict[token]["ratings"].append(review_rating)

                place_dict[token]["count"] = count

    place_df = (
        pd.DataFrame()
        .from_dict(place_dict, orient="index")
        .sort_values("count", ascending=False)
    )
    place_df["mean_rating"] = place_df["ratings"].apply(lambda x: np.mean(x))
    place_df.drop("ratings", axis=1, inplace=True)
    place_df["count_reviews"] = df.shape[0]
    place_df["count_places"] = len(df.place_id.unique())

    place_df.to_excel(table, sheet_name=tag[:30])
table.close()